In [43]:
import numpy as np
import sympy as sp
from dmbtools.geometry import utils as ug
from dmbtools.math import utils as um

#### Check S&T alpha thresholds

In [2]:
thresh_alpha = 10**-4
thresh_alpha

0.0001

In [3]:
n = np.array([1., 0., 0.]) # north vector
v = np.array([0., 0., 1.]) # vertical vector
b = ug.gibbs(thresh_alpha, v, n) # rotate the north vector around the vertical vector by the threshold angle
print(b)

[9.99999995e-01 9.99999998e-05 0.00000000e+00]


In [4]:
np.dot(n, b)

0.999999995

In [5]:
def isclose_comm(a, b, rtol=1e-5, atol=1e-8):
    ab = np.stack((a, b))
    return np.abs(a - b) <= (atol + rtol * ab.max(axis=0))

In [6]:
def allclose_comm(a, b, rtol=1e-5, atol=1e-8):
    return isclose_comm(a, b, rtol, atol).all()

In [7]:
def rvt2matrix(x, y, s, t, rtol=1e-05, atol=1e-08):
    """
    http://people.eecs.berkeley.edu/~wkahan/MathH110/Cross.pdf
    Paragraph 7: Constructing Rotations out of Reflections in a Euclidean Space of any Dimension ≥ 2
    """
    if np.any(np.linalg.norm(np.array([x, y, s, t]), axis=1) == 0):
        raise ValueError('Input vectors can not be zero length')
    if not allclose_comm(np.linalg.norm(x), np.linalg.norm(y), rtol=rtol, atol=atol):
        raise ValueError('Input vectors x, y must be the same length')
    if not allclose_comm(np.linalg.norm(s), np.linalg.norm(t), rtol=rtol, atol=atol):
        raise ValueError('Input vectors s, t must be the same length')
    if not allclose_comm(np.dot(s, x), np.dot(t, y)):
        raise ValueError('The angle between input vectors s and x must equal the anlge between t and y')
        
    I = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
    w = x - y
    W = I - 2.0 * (np.outer(w, w) / w.dot(w))
    if allclose_comm(W.dot(s), t, rtol=rtol, atol=atol):
        h = np.cross(y, t)
    else:
        h = W.dot(s - t)
    H = I - 2.0 * (np.outer(h, h) / h.dot(h))
    R = W.dot(H) #R = H.dot(W)
    
    def _r(nev):
        nev = np.atleast_2d(nev)
        return R.dot(nev.T).T
    return _r

In [8]:
def rvt2matrix3(x, y, s, t, rtol=1e-05, atol=1e-08):
    """
    http://people.eecs.berkeley.edu/~wkahan/MathH110/Cross.pdf
    Paragraph 7: Constructing Rotations out of Reflections in a Euclidean Space of any Dimension ≥ 2
    """
    xyst = np.array([x, y, s, t])
    if np.any(np.linalg.norm(xyst, axis=1) == 0):
        raise ValueError('Input vectors can not be zero length')
    xyst = um.normalized(xyst)
    if not allclose_comm(xyst[2].dot(xyst[0]), xyst[3].dot(xyst[1])):
        raise ValueError('The angle between input vectors s and x must equal the anlge between t and y')
        
    I = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
    w = xyst[0] - xyst[1]
    W = I - 2.0 * (np.outer(w, w) / w.dot(w))
    
    if allclose_comm(W.dot(xyst[2]), xyst[3], rtol=rtol, atol=atol):
        h = np.cross(xyst[1], xyst[3])
    else:
        h = W.dot(xyst[2] - xyst[3])
    
    H = I - 2.0 * (np.outer(h, h) / h.dot(h))
    R = W.dot(H) #R = H.dot(W)
    print(R)
    
    # Orthonormalize the Rotation Matrix in case our initial angles are not exaclty the same
    uu, ss, vh = np.linalg.svd(R, full_matrices=True)
    Ro = uu.dot(vh.conj().T)
    print(Ro)
    
    def _r(nev):
        nev = np.atleast_2d(nev)
        return Ro.dot(nev.T).T
    return _r

In [9]:
def rvt2matrix2(x, y, rtol=1e-05, atol=1e-08):
    """
    http://people.eecs.berkeley.edu/~wkahan/MathH110/Cross.pdf
    Paragraph 7: Constructing Rotations out of Reflections in a Euclidean Space of any Dimension ≥ 2
    """
    xy = np.array([x, y])
    if np.any(np.linalg.norm(xy, axis=1) == 0):
        raise ValueError('Input vectors can not be zero length')
    xyst = um.normalized(xy)
        
    I = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
    w = xy[0] - xy[1]
    W = I - 2.0 * np.outer(w, w) / np.dot(w, w)
    R = W.T
    
    def _r(nev):
        nev = np.atleast_2d(nev)
        return np.dot(R, nev.T).T
    return _r

In [10]:
x = np.array([1., 0., 0.])
y = np.array([0., 0., 0.])
a = np.array([x, y])
a

array([[1., 0., 0.],
       [0., 0., 0.]])

In [11]:
np.any(np.linalg.norm(a, axis=1))

True

In [12]:
np.allclose(np.linalg.norm(x), np.linalg.norm(x))

True

In [13]:
np.allclose(np.linalg.norm(x), np.linalg.norm(y))

False

In [14]:
np.allclose(np.linalg.norm(x), np.linalg.norm(x)) & np.allclose(np.linalg.norm(x), np.linalg.norm(y))

False

In [15]:
x = np.array([ 0.,  1.,  0.]) # E
y = np.array([ 1.,  0.,  0.]) # N
s = np.array([ 0.,  0.,  1.]) # V
t = np.array([ 0., -1.,  0.]) # W

i = np.array([-1.,  0.,  0.]) # S
f = np.array([ 0.,  0.,  1.]) # -V

In [16]:
not np.allclose([np.dot(s, x)], [np.dot(t, y)])

False

In [17]:
I = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
w = x - y
W = I - 2.0 * (np.outer(w, w) / w.dot(w)) #W = I - 2.0 * (np.outer(w, w) / np.dot(w, w))

In [18]:
W

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [19]:
h = W.dot(s - t)
h

array([1., 0., 1.])

In [20]:
h = np.matmul(W, (s - t))
h

array([1., 0., 1.])

In [21]:
h = W @ (s - t)
h

array([1., 0., 1.])

In [22]:
np.cross(x ,t)

array([ 0.,  0., -0.])

In [23]:
H = I - 2.0 * (np.outer(h, h) / h.dot(h)) #H = I - 2.0 * (np.outer(h, h) / np.dot(h, h))
H

array([[ 0.,  0., -1.],
       [ 0.,  1.,  0.],
       [-1.,  0.,  0.]])

In [48]:
W.dot(H) # <--- this gives the correct rotation

array([[ 0.,  1.,  0.],
       [ 0.,  0., -1.],
       [-1.,  0.,  0.]])

In [24]:
R = H.dot(W) #R = W.dot(H)
R

array([[ 0.,  0., -1.],
       [ 1.,  0.,  0.],
       [ 0., -1.,  0.]])

In [45]:
R = np.matmul(H, W)
R

array([[ 0.,  0., -1.],
       [ 1.,  0.,  0.],
       [ 0., -1.,  0.]])

In [26]:
R = W @ H
R

array([[ 0.,  1.,  0.],
       [ 0.,  0., -1.],
       [-1.,  0.,  0.]])

In [56]:
H @ W

array([[ 0.,  0., -1.],
       [ 1.,  0.,  0.],
       [ 0., -1.,  0.]])

In [47]:
sp.Matrix(H) * sp.Matrix(W)

Matrix([
[  0,    0, -1.0],
[1.0,    0,    0],
[  0, -1.0,    0]])

In [27]:
np.matmul(R, i)

array([0., 0., 1.])

In [28]:
R @ i

array([0., 0., 1.])

In [29]:
R.dot(i)

array([0., 0., 1.])

In [30]:
R.dot(np.atleast_2d(i).T).T

array([[0., 0., 1.]])

In [31]:
print(np.array([x, y, s, t]))
np.any(np.linalg.norm(np.array([x, y, s, t]), axis=1) == 0)

[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0. -1.  0.]]


False

In [32]:
np.linalg.norm(np.array([x, y, s, t]), axis=1) == 0

array([False, False, False, False])

In [33]:
RR = rvt2matrix(x, y, s, t)

In [34]:
RR([i])

array([[0., 0., 1.]])

In [35]:
a = np.array([i, x, y, s, t, f])
a

array([[-1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 0.,  0.,  1.]])

In [36]:
RR(a)

array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0., -1.],
       [ 0., -1.,  0.],
       [-1.,  0.,  0.],
       [ 0., -1.,  0.]])

In [37]:
R.dot(np.atleast_2d(a).T).T

array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0., -1.],
       [ 0., -1.,  0.],
       [-1.,  0.,  0.],
       [ 0., -1.,  0.]])

In [38]:
R3 = rvt2matrix3(x, y, s, t)

[[ 0.  1.  0.]
 [ 0.  0. -1.]
 [-1.  0.  0.]]
[[ 0.  1.  0.]
 [ 0.  0. -1.]
 [-1.  0.  0.]]


In [39]:
R3(a)

array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0., -1.],
       [ 0., -1.,  0.],
       [-1.,  0.,  0.],
       [ 0., -1.,  0.]])

#### two vector

In [40]:
a

array([[-1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0., -1.,  0.],
       [ 0.,  0.,  1.]])

In [41]:
R2 = rvt2matrix2(x, y)

In [42]:
R2(a)

array([[ 0., -1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [-1.,  0.,  0.],
       [ 0.,  0.,  1.]])